![人工智慧 - 自由團隊](https://raw.githubusercontent.com/chenkenanalytic/img/master/af/aifreeteam.png)


<center>Welcome to the course《Python: from business analytics to Artificial Intelligence》by AI . FREE Team.</center>
<br>
<center>歡迎大家來到 AI . FREE Team 《Python 從商業分析到人工智慧》的 AI 基礎教學 - Lesson 05 Convolution Neural Network (3-05)。 </center>
<br>

<center>(Author: Chen Ken；Date of published: 2021//；AI . FREE Team Website: https://aifreeblog.herokuapp.com/)</center>

# 卷積神經網路


在上一篇教學，我們使用深度神經網路框架 (tensorflow & Keras)，架構出簡易的圖片辨識器，但是針對模型的辨識準確率及訓練效率，似乎不是太過理想，因此針對影像相關 AI 演算法的開發，我們將使用卷積神經網路優化模型架構。

## STEP 0. 註冊 Kaggel 帳號 & 下載貓狗辨識資料集

Kaggle 貓狗競賽網址： https://www.kaggle.com/c/dogs-vs-cats

※ 下載完資料集，並上傳、解壓縮 (或是透過google drive連結，每次使用此教學即可避免重複上傳)

In [ ]:
### 資料上傳時間，依電腦網速而定
from google.colab import files
uploaded = files.upload()

In [ ]:
### 解壓縮資料集，路徑請依據實際情況調整
!unzip "/content/dogs-vs-cats.zip"
!unzip "/content/train.zip"
!unzip "/content/test1.zip"

## STEP 1. 資料集基礎整理

In [ ]:
# 建立資料夾
!mkdir -p training/cats
!mkdir -p training/dogs
!mkdir -p valid/cats
!mkdir -p valid/dogs

In [ ]:
# 資料分類 (tensorflow 內建模組 - 透過資料夾分類作為 label)
import os
os.chdir("train")
!cp cat.?.jpg cat.??.jpg cat.???.jpg cat.????.jpg ../training/cats/
!cp dog.?.jpg dog.??.jpg dog.???.jpg dog.????.jpg ../training/dogs/
!cp cat.1????.jpg ../valid/cats/
!cp dog.1????.jpg ../valid/dogs/
os.chdir("/content")

## STEP 2. 認識卷積 (convolution)

卷積矩陣在電腦視覺領域，作為圖像處理的工具，可進行圖片的資訊擷取，例如：邊緣偵測(圖片的輪廓)、色彩的過濾等等，我們先來認識卷積矩陣如何進行邊緣偵測。

In [ ]:
from PIL import Image
from numpy import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import signal

### 圖片位置 (可嘗試不同圖片)
img_path = '/content/test1/10019.jpg'

### 讀取圖片
image = Image.open(img_path)

### 轉為黑白圖片
image_b = image.convert('1')

### 將圖像矩陣，轉換成 2D 的圖像矩陣
A = array(image_b)
new_image = empty((A.shape[0],A.shape[1]),None)
for i in range(len(A)):
    for j in range(len(A[i])):
        if A[i][j]==True:
            new_image[i][j]=0
        else:
            new_image[i][j]=1

plt.imshow(new_image,cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# 卷積矩陣 (可更換矩陣中的數值，看看運算結果有何不同)
k = [
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
    ]

### 顯示原始圖片 - 黑白圖片
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.imshow(new_image,cmap='gray')
plt.axis('off')
plt.title('Gray Image')

### 顯示邊緣偵測後的圖片
plt.subplot(1,2,2)

### 黑白圖片與卷積矩陣進行運算
edge = signal.convolve2d(new_image, k, boundary='symm', mode='same')
plt.imshow(edge,cmap='gray')
plt.axis('off')
plt.title('Edge Detection')

### ※【重要觀察】卷積矩陣

從上面的範例，可以發現卷積矩陣能夠透過運算進行圖片資訊的擷取，根據此特性，AI 研究者便應用了卷積矩陣於神經網路中，透過圖像任務的學習，讓梯度去更新卷積矩陣的權重，故稱作卷積神經網絡，也因為卷積矩陣能夠有效地去處理圖片的資訊，甚至能減少神經網路的模型參數，因此奠定卷積神經網路在 AI 電腦視覺領域的地位。

接下來我們使用卷積神經網路來進行貓狗辨識。

## STEP 3. 引入框架與開發套件

In [ ]:
import tensorflow as tf

### flatten 轉換圖片為矩陣；Dense 為一般類神經網路層
from tensorflow.python.keras.layers import Flatten, Dense

### 使用 ImageDataGenerator 作為資料擴增用途
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

## STEP 4. 基礎參數設定

In [ ]:
# 圖片檔案路徑 (依據實際開發過程調整路徑)
train_data_path = "training"
valid_data_path = "valid"

# 影像大小
image_size = (224, 224)

# 每一次訓練的圖片數量 (若 batch_size 太大，會導致 GPU 記憶體不夠用)
batch_size = 64

# Epoch 數
epochs = 20

## STEP 5. 資料擴增 Data Augmentation

In [ ]:
# 透過資料擴增，增加訓練及驗證的資料集
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_batches = train_datagen.flow_from_directory(train_data_path,
                                                  target_size=image_size,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=batch_size)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(valid_data_path,
                                                  target_size=image_size,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=batch_size)

## STEP 6. 檢視資料集及標籤

In [ ]:
for name, label in train_batches.class_indices.items():
    print('類別編號：【{}】 為 {}'.format(label, name))

## STEP 7. 建構卷積神經模型

使用 keras 建構卷積神經模型

In [ ]:
### 定義 sequential 模型，使開發者可透過一層一層地堆疊模型層數
model = tf.keras.models.Sequential()

### 圖片的大小為 224 * 224 * 3 (後面的乘三為 rgb)
model.add(tf.keras.Input(shape=(image_size[0],image_size[1],3)))

### 建立一連串卷積層及池化層
model.add(tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
# model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(64, kernel_size = (3,3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
# model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(128, kernel_size = (3,3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
model.add(tf.keras.layers.Dropout(0.25))

### 轉成全連接層
model.add(Flatten())

### 第一層神經網絡 (512) 個神經元)
model.add(Dense(512))

### 使用 relu 作為啟動函數
model.add(tf.keras.layers.Activation(tf.keras.activations.relu))

### 使用 dropout 避免 overfitting
model.add(tf.keras.layers.Dropout(0.5))

### 第三層神經網絡 (2 個神經元 → 因為將預測出貓或狗的機率)
model.add(Dense(2))

### 多類別辨識使用 softmax 作為啟動函數
model.add(tf.keras.layers.Activation(tf.keras.activations.softmax))

### 檢視模型的架構
model.summary()

## STEP 8. 設定優化器與開始訓練

In [ ]:
### 使用 Adam 優化器；損失函數使用分類交叉熵
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 開始訓練
model.fit_generator(train_batches,
                    steps_per_epoch = train_batches.samples // batch_size,
                    validation_data = valid_batches,
                    validation_steps = valid_batches.samples // batch_size,
                    epochs = epochs)

## STEP 9. 模型驗證與辨識

可嘗試使用 test1 資料夾的圖片做驗證，或上傳貓狗圖片嘗試做辨識。

In [ ]:
from tensorflow.python.keras.preprocessing import image
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

### 圖片位置
img_path = '/content/test1/10315.jpg'

### 讀取圖片
img = image.load_img(img_path, target_size=(224, 224))

### 轉換圖片為矩陣
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)

### 進行辨識
recog = model.predict(x)[0]

### 查看辨識結果
print("判別為貓的機率為： ", recog[0] * 100, " %")
print("判別為狗的機率為： ", recog[1] * 100, " %")

image = mpimg.imread(img_path)
plt.imshow(image)
plt.axis('off')
plt.show()

if recog[0] > recog[1]:
  print("因此合理推斷，圖片應為貓的圖片！")
else:
  print("因此合理推斷，圖片應為狗的圖片！")

#結論與發現

透過開發卷積神經網路，可以發現模型的訓練效率以及參數數量大幅減少 (上一篇教學：近4億個參數；本篇僅4千多萬)，

在相差近十倍的模型參數下，卷積神經網路展現了更佳的準確率！

因此我們可以確知卷積神經網路在現今 AI 電腦視覺的地位，

此外，談到電腦視覺，多數資料科學家的起手任務多為 MNIST (手寫阿拉伯數字辨識)，

link: http://yann.lecun.com/exdb/mnist/

感興趣的讀者，可以嘗試將本篇範例嘗試修改為 MNIST 的辨識任務，

下一篇教學，我們將帶領讀者實作：繁體中文手寫辨識。

# 更深入的教學與專案實作

如果你/妳對於深入開發 AI 技術的專案有興趣，想嘗試更多、更有趣、更扎實的實務AI技術專案，

歡迎參考我們從0到1的AI技術課程：[《學習 AI 一把抓：點亮人工智慧技能樹》](https://hahow.in/cr/slashie-ai-free-team)！

# 課程文件

### AI Foundations 課程清單
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 00 Preface 課程前言</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 01 MP model</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 02 Logistic Regression</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 03 Neural Network</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 04 Deep Neural Network</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 05 Convolution Neural Network</a> (We are here now! --本篇課程--)
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 06 Handwriting Traditional Chinese Characters Recognition</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 07 Sentiment Analysis</a>